In [1]:
import psycopg
import pandas as pd
from datetime import datetime
from io import StringIO

In [2]:
file_path = r'C:\Users\tom_r\Desktop\santander-bikes\data\387JourneyDataExtract01Jan2024-14Jan2024.csv'

df = pd.read_csv(file_path)
df

,Number,Start date,Start station number,Start station,End date,End station number,End station,Bike number,Bike model,Total duration,Total duration (ms)
0,136666627,2024-01-14 23:59,1108,"North Wharf Road, Paddington",2024-01-15 00:06,3423,"Maida Vale, Maida Vale",53020,CLASSIC,6m 47s,407799
1,136666625,2024-01-14 23:57,3447,"Gloucester Road (North), Kensington",2024-01-15 00:05,1214,"Kensington Olympia Station, Olympia",54559,CLASSIC,8m 1s,481276
2,136666626,2024-01-14 23:57,1090,"Warren Street Station, Euston",2024-01-15 00:02,200005,"New Cavendish Street, Marylebone",52133,CLASSIC,5m 11s,311788
3,136666622,2024-01-14 23:56,200058,"Northdown Street, King's Cross",2024-01-15 00:06,2687,"Brick Lane Market, Shoreditch",60341,PBSC_EBIKE,10m 14s,614030
4,136666623,2024-01-14 23:56,1052,"Soho Square , Soho",2024-01-15 00:23,1150,"Queensway, Kensington Gardens",55212,CLASSIC,27m 1s,1621583
...,...,...,...,...,...,...,...,...,...,...,...
212577,136450337,2024-01-01 00:03,1034,"Bruton Street, Mayfair",2024-01-01 01:29,3452,"Panton Street, West End",60538,PBSC_EBIKE,1h 26m 2s,5162949
212578,136450332,2024-01-01 00:01,300235,"York Way, Kings Cross",2024-01-01 00:31,200212,"Blackfriars Station, St. Paul's",54136,CLASSIC,29m 54s,1794054
212579,136450333,2024-01-01 00:01,300235,"York Way, Kings Cross",2024-01-01 00:31,200212,"Blackfriars Station, St. Paul's",57461,CLASSIC,29m 43s,1783017
212580,136450334,2024-01-01 00:01,300235,"York Way, Kings Cross",2024-01-01 00:30,200212,"Blackfriars Station, St. Paul's",21426,CLASSIC,29m 23s,1763701


In [40]:
# Connect to db
conn = psycopg.connect(
    dbname="transport",
    user="postgres",             
    password="Ghcrlposk8g!",           # REMOVE PASSWORD BEFORE LOADING TO GITHUB!
    host="127.0.0.1",
    port="5432"
)
conn.autocommit = True

('transport',)
('1', 'River Street, Clerkenwell', '-0.109971', '51.5292', '531202.52', '182832.02')


In [42]:
file_name = file_path.split('\\')[-1]
file_length = len(df)


def log_start(cur):
    starttime = datetime.now()
    cur.execute("INSERT INTO etl.load_history (filename, file_row_count, load_started_at) VALUES (%s, %s, %s)",
    (file_name, file_length, starttime))
    return starttime

def log_end(cur, starttime, success, message = 'Success'):
    print(starttime)
    finish = datetime.now()
    cur.execute("UPDATE etl.load_history SET load_finished_at = %s, status = %s, error_message = %s WHERE load_started_at = %s", (finish, success, message, starttime))
    


In [43]:
# CREATE TABLE raw.journeys_raw (
# journey_id			int,
# start_date			timestamp,
# start_station_id	text,
# start_station		text,
# end_date			timestamp,
# end_station_id		text,
# end_station			text,
# bike_number			int,
# bike_model			text,
# total_duration		text,
# total_duration_ms	int
# )

In [44]:
with conn.cursor() as cur:
    started = log_start(cur)
    try:
        # write df in memory
        buf = StringIO()
        df.to_csv(buf, index=False, header=False)
        buf.seek(0)

        # copy
        with cur.copy("COPY raw.journeys_raw (journey_id, start_date, start_station_id, start_station, end_date, end_station_id, end_station, bike_number, bike_model, total_duration, total_duration_ms) FROM STDIN WITH (FORMAT CSV)") as cp:
            cp.write(buf.getvalue())

        success = True
        msg = "Load successful"
    except Exception as e:
        success = False
        msg = f"Load failed: {e}"

    log_end(cur, started, success, msg)
    conn.commit()

conn.close()

2025-09-06 10:54:20.275478


In [45]:
print(len(df))

212582
